# 簡單用法

In [1]:
import jieba

def generate_ngrams(words, n):
    # 建立空的 n-grams 列表
    ngrams = []

    # 迭代詞彙列表中的每個詞
    for i in range(len(words) - n + 1):
        # 新增下一個 n-gram
        ngrams.append(tuple(words[i:i+n]))

    return ngrams

# 範例輸入
text = "我喜歡閱讀書籍，也喜歡使用電腦來學習新的知識"

# 使用 jieba 進行斷詞
words = jieba.lcut(text)

# 產生 bi-grams
bigrams = generate_ngrams(words, 2)

# 輸出結果
for bigram in bigrams:
    print(bigram)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\darren\AppData\Local\Temp\jieba.cache
Loading model cost 0.320 seconds.
Prefix dict has been built succesfully.


('我', '喜歡')
('喜歡', '閱讀')
('閱讀', '書籍')
('書籍', '，')
('，', '也')
('也', '喜歡')
('喜歡', '使用')
('使用', '電腦')
('電腦', '來')
('來', '學習')
('學習', '新')
('新', '的')
('的', '知識')


# 計算下一個字出現的機率

In [2]:
# 假設我們已經有了一個已斷詞的文本列表
tokens = ['我', '愛', '吃', '蘋果', '，', '我', '也', '愛', '吃', '橘子']

# 建立一個預設為空列表的詞典
ngram_dict = {}

# 指定我們要使用的 N-gram 的 N 值
N = 2

# 遍歷所有的詞彙
for i in range(len(tokens)-N):
    # 得到 N-gram 和它的下一個詞彙
    ngram = tuple(tokens[i:i+N])
    next_token = tokens[i+N]

    # 更新詞典
    if ngram not in ngram_dict:
        ngram_dict[ngram] = {}
    if next_token not in ngram_dict[ngram]:
        ngram_dict[ngram][next_token] = 0
    ngram_dict[ngram][next_token] += 1

# 轉換次數為機率: dict.items() 會回傳一個 (key, value) 的 tuple
for ngram, next_tokens in ngram_dict.items():
    '''
    ngram 是 tuple，
    next_tokens 是一個 dict，key 是下一個詞彙，value 是出現的次數。

    ngram, next_tokens 的內容如下：
    ('我', '愛') {'吃': 1}
    ('愛', '吃') {'蘋果': 1, '橘子': 1}
    ('吃', '蘋果') {'，': 1}
    ('蘋果', '，') {'我': 1}
    ('，', '我') {'也': 1}
    ('我', '也') {'愛': 1}
    ('也', '愛') {'吃': 1}
    '''

    '''
    next_tokens.values() 會回傳一個 dict_values 物件，裡面是所有的次數:
    dict_values([1])
    dict_values([1, 1])
    dict_values([1])
    dict_values([1])
    dict_values([1])
    dict_values([1])
    dict_values([1])
    '''
    total_count = sum(next_tokens.values())

    # 將次數轉換為機率: dict.items() 會回傳一個 (key, value) 的 tuple
    for next_token, count in next_tokens.items():
        '''
        ngram 是 tuple，
        next_tokens 是一個 dict，key 是下一個詞彙，value 是出現的次數。
        '''
        ngram_dict[ngram][next_token] = count / total_count

# 假設我們想要預測愛吃之後的下一個詞彙
ngram = ('愛', '吃')

# 從詞典中取得所有可能的下一個詞彙和它們的機率
next_tokens_probs = ngram_dict[ngram]
'''
next_tokens_probs 的內容如下：
{'蘋果': 0.5, '橘子': 0.5}
'''

# 將它們按照機率排序，取前 k 個
k = 2
top_k_next_tokens = sorted(
    next_tokens_probs.items(), # dict.items() 會回傳一個 (key, value) 的 tuple
    key=lambda x: x[1], 
    reverse=True
)[:k]

print(top_k_next_tokens)

[('蘋果', 0.5), ('橘子', 0.5)]


In [3]:
'''
# 雅卡爾指數
https://zh.wikipedia.org/zh-tw/雅卡尔指数
'''

# 使用 N-gram 計算兩個句子的相似度
def jaccard_similarity(s1, s2, N):
    # 使用 jieba 進行斷詞
    words1 = jieba.lcut(s1)
    words2 = jieba.lcut(s2)

    # 產生 N-grams
    ngrams1 = generate_ngrams(words1, N)
    ngrams2 = generate_ngrams(words2, N)

    # 計算兩個句子的相似度
    common_ngrams = set(ngrams1) & set(ngrams2) # 交集
    ngrams_union = set(ngrams1) | set(ngrams2) # 聯集

    # 處理分母為 0 的情況
    if len(ngrams_union) == 0:
        return 0

    # 透過計算共同 N-grams 的數量來計算相似度
    similarity = len(common_ngrams) / len(ngrams_union)

    return similarity



# 範例
s1 = "在遙遠的東方，有一個美麗的國度，那裡山清水秀，人民勤勞智慧。"
s2 = "在很遠的東方，有一個美麗的國家，那裡山明水秀，人民勤勞聰明。"

# 使用 N-gram 來計算兩句話的相似度
similarity = jaccard_similarity(s1, s2, 2)
print(similarity)

0.3448275862068966
